# Information

You can run the entire script by press **CTRL+F9**.

For more helpful scripts, check out the quick commands.
*   [Robohood | Quick commands](https://colab.research.google.com/drive1WsbRD8Rlz_ceSoGQzeGPZX5bNeRLLbz5?usp=sharing)
*  [Cancel all pending orders](https://colab.research.google.com/drive/1WsbRD8Rlz_ceSoGQzeGPZX5bNeRLLbz5#scrollTo=ElxYAXz3LWR5&line=1&uniqifier=1)
*  [Close all positions](https://colab.research.google.com/drive/1WsbRD8Rlz_ceSoGQzeGPZX5bNeRLLbz5#scrollTo=LD2LSDnqOlFe&line=1&uniqifier=1)

# Application code
Only needs to be run once per session.

In [ ]:
#%pip install robin_stocks

In [ ]:
from robin_stocks import robinhood as r
import itertools
from datetime import datetime
import getpass
import sys
import time


## Funcitons

In [ ]:
def get_spread_cost(front_leg_option, back_leg_option, params):
  spread_cost = -1
  front_price = front_leg_option.get(params[0])
  back_price = back_leg_option.get(params[1])

  if (back_price and front_price):
    spread_cost = round(float(back_price) - float(front_price), 2)

  return spread_cost

In [ ]:
def get_min_order_quantity(spread):
  (front_leg_option, back_leg_option, spread_cost, profitability) = spread

  back_size = back_leg_option.get("ask_size")
  front_size = front_leg_option.get("bid_size")

  if (front_size and back_size):
    return min(int(front_size), int(back_size))
  else:
    return 0

In [ ]:
def get_chance_of_profit(front_leg_option, back_leg_option):
    chance_of_profit_short = front_leg_option.get("chance_of_profit_short")
    chance_of_profit_long = back_leg_option.get("chance_of_profit_long")
    if (chance_of_profit_short and chance_of_profit_long):
        return float(chance_of_profit_short) + float(chance_of_profit_long)
    return 0

In [ ]:
def find_calendar_spreads(options, params):
  # Create a create new calendar spread for every combination of expiration dates where the short front leg expiration_date is less than the long back leg of the spread
  sorted_options = sorted(options, key=lambda x: (float(x['strike_price'])))
  options_grouped_by_strike = itertools.groupby(sorted_options, lambda x : x['strike_price'])
  today = datetime.today()

  #  using the front_leg_option and back_leg_option as the front and back legs of the trade, create a debit calendar spread
  #  with a cost of zero
  calendar_spreads = []
  for strike, options in options_grouped_by_strike:
    for (front_leg_option, back_leg_option) in itertools.combinations(options,2):
      front_expiration_date = datetime.strptime(front_leg_option["expiration_date"],'%Y-%m-%d')
      back_expiration_date = datetime.strptime(back_leg_option["expiration_date"],'%Y-%m-%d')
      optionTypeIsSame = front_leg_option["type"] == back_leg_option["type"]

      if (optionTypeIsSame and (front_expiration_date > today) and (front_expiration_date < back_expiration_date)):
        spread_cost = get_spread_cost(front_leg_option, back_leg_option, params)
        profitability = get_chance_of_profit(front_leg_option, back_leg_option)
        spread = (front_leg_option, back_leg_option,
                  spread_cost, profitability)

        calendar_spreads.append(spread)

  return calendar_spreads

In [ ]:
def get_current_price_of_symbol(symbol="SPY"):
  current_price = round(float(r.stocks.get_latest_price(symbol)[0]))
  print("{} is currently trading at: ${}".format(symbol, current_price))
  return current_price 

In [ ]:
def scan_for_options(symbol, optionType, min_strike_offset, max_strike_offset):
  if(optionType == 'both'):
    optionType = ""

  current_price = get_current_price_of_symbol(symbol)
  min_strike = current_price - min_strike_offset;
  max_strike = current_price + max_strike_offset;

  # Get the list of options available and sort by strike_price and expiration_date
  options = r.options.find_tradable_options(symbol, optionType=optionType, info=None)
  options = list(filter(lambda x: (float(x["strike_price"]) > min_strike) and (float(x["strike_price"]) < max_strike), options))
  return options

In [ ]:
def print_spread(spread):
  (front_leg_option, back_leg_option, spread_cost, profitability) = spread
  strike = float(front_leg_option.get("strike_price"))
  print("strike {} | short {} @ {} | long {} @ {} | spread: ${} | profitability: {}".format(
      strike,
      front_leg_option.get("type"),
      front_leg_option.get("expiration_date"),
      back_leg_option.get("type"),
      back_leg_option.get("expiration_date"),
      spread_cost,
      profitability))


In [ ]:
def print_spreads(spreads):
  for spread in spreads:
    print_spread(spread)

In [64]:
from time import sleep
def order_calendar_spread(spread, price=0.00, quantity=1, timeInForce='gfd', max_attempts=3, sleep_time=1):
  (front_leg_option, back_leg_option, spread_cost, profitability) = spread
  symbol = front_leg_option["symbol"]
  params = [
            {
            'expirationDate': front_leg_option['expiration_date'],
            'strike': front_leg_option['strike_price'],
            'optionType': front_leg_option['type'],
            'quantity': '1',
            'effect': 'open',
            'action': 'sell',
          },
          {
            'expirationDate': back_leg_option['expiration_date'],
            'strike': back_leg_option['strike_price'],
            'optionType': back_leg_option['type'],
            'quantity': '1',
            'effect': 'open',
            'action': 'buy',
          },
  ]
  
  print("buying {} @ ${}".format(quantity, price))

  order = r.orders.order_option_spread(direction='debit', price=price, symbol=symbol,
                                       quantity=quantity, spread=params, timeInForce=timeInForce)
  if(order):
    if (order.get("state")):
      print_spread(spread)
      return order

    if (order.get("detail")):
        print(order.get("detail"))
        return order
  else:
    attempts = 0
    while attempts < max_attempts:
      attempts += 1
      print("Something failed. Auto retry attempt {} of {}".format(attempts, max_attempts))
      order = r.orders.order_option_spread(direction='debit', price=price, symbol=symbol,
                                          quantity=quantity, spread=params, timeInForce=timeInForce)
      sleep(sleep_time)

      if attempts == max_attempts:
          print("max number of tries exceeded. Order failed because ")
          print(order.get("detail"))

  return order

In [ ]:
def place_calendar_spreads(spreads, price_type, quantity_type, timeInForce, max_quantity=250, price=0.00, quantity=1):
  results = []
  if(price_type == "set_for_all"):
    price = float(input("Please enter the PRICE for each order: ") or 0.00)

  if(quantity_type == "set_for_all"):
    quantity = int(input("Please enter the QUANTITY for each order: ") or 0)

  if (quantity_type == "use_max_available_spreads"):
    user_max_quantity = int(input("Please enter the MAX_QUANTITY for each order: ") or max_quantity)

  for spread in spreads:
    (front_leg_option, back_leg_option, spread_cost, profitability) = spread

    if(price_type == "set_each"):
      price = float(input("price (limit): ") or 0.00)

    if (price_type == "use_spread_cost"):
      price = spread_cost

    if(quantity_type == "set_each"):
      quantity = int(input("order size (quantity): ") or 1)

    if (quantity_type == "use_max_available_spreads"):
      quantity = get_min_order_quantity(spread)
      if ((quantity >= user_max_quantity)):
        quantity = user_max_quantity

    if ((quantity >= max_quantity)):
      quantity = max_quantity

    result = order_calendar_spread(spread=spread, price=price, quantity=quantity, timeInForce=timeInForce)
    results.append(result)
  return results

In [ ]:
def update_option_market_data(options):
  for item in options:
    marketData = r.options.get_option_market_data_by_id(item['id'])
    if marketData:
        item.update(marketData[0])

In [ ]:
def spread_filter(spread, max_spread_cost, min_volume, min_avalible_quantity, min_profitability, is_in_volume_range=False):
    (front_leg_option, back_leg_option, spread_cost, profitability) = spread
    front_volume = front_leg_option.get("volume")
    back_volume = back_leg_option.get("volume")

    is_less_than_max_cost = (spread_cost <= max_spread_cost)
    if (front_volume and back_volume):
        is_in_volume_range = (int(front_volume) >= min_volume) and (int(back_volume) >= min_volume)
    is_greater_than_min_avalible_quantity = get_min_order_quantity(spread) >= min_avalible_quantity

    is_greater_than_profitability = profitability >= min_profitability
    return is_less_than_max_cost and is_in_volume_range and is_greater_than_min_avalible_quantity and is_greater_than_profitability


In [ ]:
def filter_calendar_spreads(spreads, max_spread_cost, min_volume, min_avalible_quantity, min_chance_of_profitability):

  filtered_spreads = list(
      filter(lambda spread: spread_filter(spread, max_spread_cost, min_volume, min_avalible_quantity, min_chance_of_profitability), spreads))

  filtered_spreads = sorted(
      filtered_spreads, key=lambda spread: ((spread[2]), -spread[3]))
  return filtered_spreads

# Login

In [ ]:
#@title #Authenticate { vertical-output: true, display-mode: "form" }
username = "" #@param {type:"string"}
password = "" #@param {type:"string"}
login = r.login(username, password)
print(login.get("detail"))

# Scan for options

> Inputs needed: offset and max offset. By default, the scanner will analize $10 worth of different strikes. 5 above and 5 below.

* **symbol**: example "spy"
* **min strike offset**: how far below the current strike you want to look. The default is 5 dollars.
* **max strike offset**: how far above the current strike you want to look. The 
default is 5 dollars.

In [ ]:
#@title Enter option criteria { vertical-output: true, display-mode: "form" }
# Create a script that places a debit calendar spread for each tradeable option of a given symbol. The cost of the spread needs to be zero.
symbol = "SPY"  # @param {type:"string"}
optionType = "both"  # @param ["call", "put", "both"]
min_strike_offset = 2 # @param {type:"integer"}
max_strike_offset = 2  # @param {type:"integer"}

options = scan_for_options(symbol, optionType, min_strike_offset, max_strike_offset)
print("Base on user criteria, you found {} to analize!".format(len(options)))

## Get option market data


> This gets the latest market information for each option found. Data such as ask_price and bid_price are changing all the time. It's a good idea to refresh this often.

In [ ]:
update_option_market_data(options)

# Filter calendar spreads

Enter max price you want filter for each spread. This is calculated by subtracting the front leg bid price from the ask price of the back leg. ask_price - bid_price = spread_cost. The default is $0.50.

In [73]:
#@title Filter based on spread cost { run: "auto", vertical-output: true, display-mode: "form" }
max_spread_cost = .5  # @param {type:"number"}
min_volume = 0  # @param {type:"integer"}
min_avalible_quantity = 0  # @param {type:"integer"}
min_chance_of_profitability = 0 # @param {type:"number"}
# @param ["bid_price", "ask_price", "high_fill_rate_buy_price","high_fill_rate_sell_price", "low_fill_rate_buy_price", "low_fill_rate_sell_price"]
short_price_key = "ask_price"
# @param ["bid_price", "ask_price", "high_fill_rate_buy_price","high_fill_rate_sell_price", "low_fill_rate_buy_price", "low_fill_rate_sell_price"]
long_price_key = "bid_price"

params = [short_price_key, long_price_key]

calendar_spreads = find_calendar_spreads(options, params)
filtered_spreads = filter_calendar_spreads(
    calendar_spreads, max_spread_cost, min_volume, min_avalible_quantity, min_chance_of_profitability)
print("{} spreads found.".format(len(filtered_spreads)))
print_spreads(filtered_spreads)

# Create calendar spread orders

Select the number of order you want to place. Note: the max number of orders you can create in robinhood is 11.

In [ ]:
#@title How many orders do you want to place? { run: "auto", vertical-output: true, display-mode: "form" }
max_order_size = 11 #@param {type:"integer"}
top_filtered_spreads = filtered_spreads[:max_order_size] # robinhood only allows 11 orders
print_spreads(top_filtered_spreads)

In [69]:
#@title Place calendar orders based on the following parameters: { vertical-output: true, display-mode: "form" }

# @param ["set_for_all", "set_each", "use_spread_cost"]
price_type = "use_spread_cost"
quantity_type = "use_max_available_spreads" # @param ["set_for_all", "set_each", "use_max_available_spreads"]
timeInForce='gtc'  #@param ["gfd", "gtc"] 

place_trades_answer = input("Are you ready to place your trades? Enter 'yes' to continue: ")

if (place_trades_answer == "yes"):
    place_calendar_spreads(top_filtered_spreads, price_type, quantity_type, timeInForce)
else:
    print("No trades were placed.")

print("Done.")

buying 20 @ $0.01
You have more than the allowed maximum of 11 pending orders for this option chain in your brokerage account. Please cancel some before placing another order.
buying 41 @ $0.15
You have more than the allowed maximum of 11 pending orders for this option chain in your brokerage account. Please cancel some before placing another order.
buying 43 @ $0.16
You have more than the allowed maximum of 11 pending orders for this option chain in your brokerage account. Please cancel some before placing another order.
buying 1 @ $0.17
You have more than the allowed maximum of 11 pending orders for this option chain in your brokerage account. Please cancel some before placing another order.
buying 41 @ $0.18
You have more than the allowed maximum of 11 pending orders for this option chain in your brokerage account. Please cancel some before placing another order.
buying 36 @ $0.19
You have more than the allowed maximum of 11 pending orders for this option chain in your brokerage acc